In [1]:
import torch 
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable

from dataset.readVideo import DramaDataset
from NGram.tool import NGram, trigrams
from utils.tokenMaker import Lang
import pickle

useCuda = False
Variable = Variable.cuda if useCuda else Variable

In [5]:
DataDir="/home/ball/Videos/BrokeEN"
datasets = DramaDataset(basedir=DataDir,
                        maxFrame=0,
                        )
loader = torch.utils.data.DataLoader(datasets, batch_size=2, shuffle=True, num_workers=2)

Total Drama: 113


In [2]:
def getEncoder(sub):
    subVar =  Variable(torch.LongTensor(lang.sentenceToVector(sub, sos=False, eos=False)).unsqueeze(0))    
    return model.embeddings(subVar).sum(dim=1)

def getCos(s1,s2):
    eq1 = getEncoder(s1)
    eq2 = getEncoder(s2)
    data = cos(eq1, eq2).mean() #cos
    return data.cpu().data[0]

cos = nn.CosineSimilarity(dim=1)

In [ ]:
LangFile = ("NGram/model/Lang.pkl")
with open(LangFile, 'rb') as f:
    lang = pickle.load(f)
    print("Load lang model: {}. Word size: {}".format(LangFile, len(lang)))
model = torch.load("NGram/model/NGram.5.pkl").cpu()

Load lang model: NGram/model/Lang.pkl. Word size: 15512


In [44]:
sent = "ball"
sent2 = "baseball"

In [45]:
getCos(sent, sent2)

0.006567817181348801

In [27]:
getEncoder(sent).sum(dim=1).size()

torch.Size([1, 256])